In [73]:
# # Versão da Linguagem Python
# from platform import python_version
# print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

In [74]:
# Importar as bibliotecas

import pandas as pd
import numpy as np
from tqdm import tqdm
from scipy import optimize
import numpy_financial as npf
import os


In [75]:
# Versões dos pacotes usados neste jupyter notebook
# %reload_ext watermark
# %watermark -a "GEPRE" --iversions

In [76]:
# os.chdir('//sad154000/Corporativo/Gepre/Estagiarios/Validacao_Precificacao/20231113')
# os.chdir('//sad154000/Corporativo/Gepre/Estagiarios/PRECIFICACAO/outubro/02102024')

In [77]:
# Importar planilha do excel
base_dados = 'Base de Dados - Taxi Verde.xlsx'

# Importar base de produtos 
sheet1 =  pd.read_excel(base_dados, sheet_name='produtos')

# Importar curva DI/Funding
sheet2 =  pd.read_excel(base_dados, sheet_name='curva')

In [78]:
rows = len(sheet1.axes[0])

#Valores Tributos
#IR/CSLL
ir = 0.45

# PIS/COfins
pis_cofins = 0.0465

# PIS/COFINS com ISS - Aqui o PIS/COFINS com ISS é igual ao PIS/COFINS. Trata-se de um ajuste necessário para o projeto 'Táxi Verde'
iss = 0.0465

indexador = sheet1.index.values

Margem_Liquida = pd.Series(index=indexador, dtype='float64')
Tarifa_ML = pd.Series(index=indexador, dtype='float64')

In [ ]:
def ML_tarifa(tarifa):
    """
    Função para calcular a margem líquida com a TARIFA como variável
    """
    # cálculo da margem líquida com tarifa como variável
    data_inicial = sheet1.at[k,'Data inicial:']
     
    tabela_calculo = sheet2.loc[sheet2['Data']>=data_inicial]
    tabela_calculo.insert(0,'Período',range(0,len(tabela_calculo)))
    prazo = sheet1.at[k,'Prazo:']
    tabela_calculo = tabela_calculo.loc[tabela_calculo['Período']<=prazo]
    tabela_calculo.reset_index(inplace=True)
    
    plan = sheet1.at[k, 'Plan:']
    di_ano = tabela_calculo['Curva']
    di_mes = (di_ano + 1) ** (1/12) - 1
    
    desconto = pd.Series(index=tabela_calculo['Período'], dtype='float64')
    desconto[0] = 1
    for i in range(1, prazo + 1):
        desconto[i] = 1 / (1 + di_mes[i]) * (1 / desconto[i-1])
    
    tipo_taxa = sheet1.at[k, 'Tipo de Taxa:']
    taxa_nominal = sheet1.at[k, 'Taxa Nominal a.m.:'] / 100
    indexador = sheet1.at[k, '% CDI (indexador):']
    taxa_efetiva = np.where((tipo_taxa == 'Pre') | (tipo_taxa == 'Pré'), taxa_nominal, 
                           ((1 + taxa_nominal) * (1 + di_mes * indexador)) - 1)

    # Inicialização das séries
    juros = pd.Series(0, index=tabela_calculo['Período'], dtype='float64')
    saldo_devedor = pd.Series(0, index=tabela_calculo['Período'], dtype='float64')
    saldo_devedor[0] = sheet1.at[k, 'Ticket Médio (R$):']
    parcela = pd.Series(0, index=tabela_calculo['Período'], dtype='float64')
    amortizacao = pd.Series(0, index=tabela_calculo['Período'], dtype='float64')
    captacao = pd.Series(0, index=tabela_calculo['Período'], dtype='float64')
    perda_esperada = pd.Series(0, index=tabela_calculo['Período'], dtype='float64')
    
    # Variáveis operacionais (com tarifa como parâmetro)
    r_operacional = pd.Series(0, index=tabela_calculo['Período'], dtype='float64')
    c_operacional = pd.Series(0, index=tabela_calculo['Período'], dtype='float64')
    r_operacional[0] = tarifa  # AQUI É A DIFERENÇA: tarifa como variável
    
    sd_inicial = sheet1.at[k, 'Ticket Médio (R$):']
    c_correspondente = sheet1.at[k, 'Custo com Correspondente(%):']
    c_liberacao = sheet1.at[k, 'Custo de Liberação (R$):']
    c_operacional[0] = -(sd_inicial * c_correspondente + c_liberacao)
    
    # Cálculos iniciais (período 0)
    pis_cofins_iss = pd.Series(0, index=tabela_calculo['Período'], dtype='float64')
    pis_cofins_iss[0] = -((captacao[0] + juros[0]) * pis_cofins + (r_operacional[0] * iss))
    
    lair = pd.Series(0, index=tabela_calculo['Período'], dtype='float64')
    lair[0] = (juros[0] + captacao[0] + perda_esperada[0] + c_operacional[0] + 
               r_operacional[0] + pis_cofins_iss[0])
    
    imposto_renda = pd.Series(0, index=tabela_calculo['Período'], dtype='float64')
    imposto_renda[0] = -lair[0] * ir
    
    lucro = pd.Series(0, index=tabela_calculo['Período'], dtype='float64')
    lucro[0] = lair[0] + imposto_renda[0]
    
    funding = di_mes
    carência = sheet1.at[k, 'Carência (meses):']
    
    # Loop principal
    for i in range(1, prazo + 1):
        juros[i] = saldo_devedor[i-1] * taxa_efetiva[i]
        captacao[i] = -saldo_devedor[i-1] * funding[i]
        
        if i <= carência:
            if plan == 'Antecipações':
                parcela[i] = 0
                amortizacao[i] = 0
                saldo_devedor[i] = saldo_devedor[i-1] + juros[i]
            else:
                parcela[i] = juros[i]
                amortizacao[i] = 0
                saldo_devedor[i] = saldo_devedor[i-1] + juros[i] - parcela[i]
        else:
            if plan == 'Price':
                parcela[i] = npf.pmt(taxa_efetiva[i], prazo - tabela_calculo['Período'][i-1], -saldo_devedor[i-1])
                amortizacao[i] = parcela[i] - juros[i]
                saldo_devedor[i] = saldo_devedor[i-1] + juros[i] - parcela[i]
            elif plan == 'Limites':
                if i == prazo:
                    amortizacao[i] = sd_inicial
                    saldo_devedor[i] = saldo_devedor[i-1] - amortizacao[i]
                    parcela[i] = amortizacao[i] + juros[i]
                else:
                    amortizacao[i] = 0
                    parcela[i] = 0
                    saldo_devedor[i] = saldo_devedor[i-1]
            elif plan == 'Antecipações':
                if i == prazo:
                    amortizacao[i] = saldo_devedor[i-1]
                    saldo_devedor[i] = saldo_devedor[i-1] - amortizacao[i]
                    parcela[i] = amortizacao[i] + juros[i]
                else:
                    amortizacao[i] = 0
                    parcela[i] = 0
                    saldo_devedor[i] = saldo_devedor[i-1] + juros[i]
        
        perda_esperada[i] = -(parcela[i] * sheet1.at[k, 'Perda Esperada (%):'])
        r_operacional[i] = sheet1.at[k, 'Receita de Relacionamento (R$):']
        c_operacional[i] = -(sheet1.at[k, 'Custo de Manutenção (R$):'] + sheet1.at[k, 'Custo de Relacionamento (R$):'])
        pis_cofins_iss[i] = -((captacao[i] + juros[i]) * pis_cofins + (r_operacional[i] * iss))
        lair[i] = (juros[i] + captacao[i] + perda_esperada[i] + c_operacional[i] + 
                  r_operacional[i] + pis_cofins_iss[i])
        imposto_renda[i] = lair[i] * ir
        lucro[i] = lair[i] - imposto_renda[i]
    
    vp_lucro = (lucro * desconto).sum()
    vp_sd = (saldo_devedor * desconto).sum()
    
    margem_liquida = (vp_lucro / vp_sd) * 12
    return margem_liquida - 0.01 

In [ ]:
for k in tqdm(range(0, rows)):
    # Cálculo da margem líquida atual (com tarifa original)
    data_inicial = sheet1.at[k,'Data inicial:']
     
    tabela_calculo = sheet2.loc[sheet2['Data']>=data_inicial]
    tabela_calculo.insert(0,'Período',range(0,len(tabela_calculo)))
    prazo = sheet1.at[k,'Prazo:']
    tabela_calculo = tabela_calculo.loc[tabela_calculo['Período']<=prazo]
    tabela_calculo.reset_index(inplace=True)
    
    plan = sheet1.at[k,'Plan:']
    di_ano = tabela_calculo['Curva']
    di_mes = (di_ano+1)**(1/12)-1
    
    desconto = pd.Series(index=tabela_calculo['Período'], dtype='float64')
    desconto[0] = 1
    for i in range(1, prazo+1):    
        desconto[i] = 1/((1+di_mes[i])*(1/desconto[i-1]))
       
    tipo_taxa = sheet1.at[k,'Tipo de Taxa:']
    taxa_nominal = (sheet1.at[k,'Taxa Nominal a.m.:'])/100
    indexador = sheet1.at[k,'% CDI (indexador):']
    taxa_efetiva = np.where((tipo_taxa=='Pre') | (tipo_taxa == 'Pré'), taxa_nominal, 
                           ((1+taxa_nominal)*(1+di_mes*indexador))-1)  
    
    # Cálculo com tarifa original para obter ML atual
    juros = pd.Series(index=tabela_calculo['Período'], dtype='float64')
    juros[0] = 0
    
    sd_inicial = sheet1.at[k,'Ticket Médio (R$):']
    
    saldo_devedor = pd.Series(index=tabela_calculo['Período'], dtype='float64')
    saldo_devedor[0] = sd_inicial
    
    parcela = pd.Series(index=tabela_calculo['Período'], dtype='float64')
    parcela[0] = 0
    
    amortizacao = pd.Series(index=tabela_calculo['Período'], dtype='float64')
    amortizacao[0] = 0
    
    captacao = pd.Series(index=tabela_calculo['Período'], dtype='float64')
    captacao[0] = 0
    
    perda_esperada = pd.Series(0, index=tabela_calculo['Período'], dtype='float64')
    
    tarifa = sheet1.at[k,'Tarifa (R$):']
    r_relacionamento = sheet1.at[k,'Receita de Relacionamento (R$):']
    c_liberacao = sheet1.at[k,'Custo de Liberação (R$):']
    c_manutencao = sheet1.at[k,'Custo de Manutenção (R$):']
    c_relacionamento = sheet1.at[k,'Custo de Relacionamento (R$):']
    c_correspondente = sheet1.at[k,'Custo com Correspondente(%):']
    
    r_operacional = pd.Series(0, index=tabela_calculo['Período'], dtype='float64')
    c_operacional = pd.Series(0, index=tabela_calculo['Período'], dtype='float64')
    r_operacional[0] = tarifa
    c_operacional[0] = -(sd_inicial*c_correspondente + c_liberacao)
    
    pis_cofins_iss = pd.Series(0, index=tabela_calculo['Período'], dtype='float64')
    pis_cofins_iss[0] = -((captacao[0]+juros[0])*pis_cofins+(r_operacional[0]*iss))
    
    lair = pd.Series(index=tabela_calculo['Período'], dtype='float64')
    lair[0] = (juros[0] + captacao[0] + perda_esperada[0] + c_operacional[0] + r_operacional[0] + pis_cofins_iss[0])
    
    imposto_renda = pd.Series(index=tabela_calculo['Período'], dtype='float64')
    imposto_renda[0] = -lair[0]*ir
    
    lucro = pd.Series(index=tabela_calculo['Período'], dtype='float64')
    lucro[0] = lair[0]+imposto_renda[0]
    
    funding = di_mes
    carência = sheet1.at[k,'Carência (meses):']
    
    for i in range(1, prazo+1):
        juros[i] = saldo_devedor[i-1]*taxa_efetiva[i]
        captacao[i] = -saldo_devedor[i-1]*funding[i]
        
        if i <= carência:
            if plan == 'Antecipações':
                parcela[i] = 0
                amortizacao[i] = 0
                saldo_devedor[i] = saldo_devedor[i-1]+juros[i]
            else:
                parcela[i] = juros[i]
                amortizacao[i] = 0
                saldo_devedor[i] = saldo_devedor[i-1]+juros[i]-parcela[i]
        else:
            if plan == 'Price':
                parcela[i] = npf.pmt(taxa_efetiva[i], prazo-tabela_calculo['Período'][i-1], -saldo_devedor[i-1])
                amortizacao[i] = parcela[i]-juros[i]
                saldo_devedor[i] = saldo_devedor[i-1]+juros[i]-parcela[i]
            elif plan == 'Limites':
                if i == prazo:
                    amortizacao[i] = sd_inicial
                    saldo_devedor[i] = saldo_devedor[i-1]-amortizacao[i]
                    parcela[i] = amortizacao[i]+juros[i]
                else:
                    amortizacao[i] = 0
                    parcela[i] = 0
                    saldo_devedor[i] = saldo_devedor[i-1]
            elif plan == 'Antecipações':
                if i == prazo:
                    amortizacao[i] = saldo_devedor[i-1]
                    saldo_devedor[i] = saldo_devedor[i-1]-amortizacao[i]
                    parcela[i] = amortizacao[i]+juros[i]
                else:
                    amortizacao[i] = 0
                    parcela[i] = 0
                    saldo_devedor[i] = saldo_devedor[i-1]+juros[i]
                
        perda_esperada[i] = -(parcela[i]*sheet1.at[k,'Perda Esperada (%):'])
        r_operacional[i] = r_relacionamento
        c_operacional[i] = -(c_manutencao+c_relacionamento)
        pis_cofins_iss[i] = -((captacao[i]+juros[i])*pis_cofins+(r_operacional[i]*iss))
        lair[i] = juros[i] + captacao[i] + perda_esperada[i] + c_operacional[i] + r_operacional[i] + pis_cofins_iss[i]
        imposto_renda[i] = lair[i]*ir
        lucro[i] = lair[i] - imposto_renda[i]
    
    vp_lucro = (lucro*desconto).sum()
    vp_sd = (saldo_devedor*desconto).sum()
    
    # Calculo Margem Líquida atual
    margem_liquida = (vp_lucro/vp_sd)*12
    Margem_Liquida[k] = margem_liquida
    
    # Calcular tarifa necessária para ML = 1% usando otimização
    try:
        tarifa_ml = optimize.newton(ML_tarifa, tarifa)  # Usa tarifa atual como valor inicial
        Tarifa_ML[k] = tarifa_ml
    except:
        # Em caso de erro na otimização, usar método alternativo
        try:
            tarifa_ml = optimize.fsolve(ML_tarifa, tarifa)[0]
            Tarifa_ML[k] = tarifa_ml
        except:
            Tarifa_ML[k] = np.nan

100%|██████████| 22/22 [00:01<00:00, 15.26it/s]


In [ ]:
sheet1['ML_Atual'] = Margem_Liquida
sheet1['Tarifa_ML_1%'] = Tarifa_ML
sheet1['Diferenca_Tarifa'] = Tarifa_ML - sheet1['Tarifa (R$):']

sheet1.to_excel('porrAeddc.xlsx', index = False)

In [82]:
print("Cálculo concluído!")
print(f"Margem Líquida atual média: {Margem_Liquida.mean():.4f}")
print(f"Tarifa média para ML 1%: {Tarifa_ML.mean():.2f}")


Cálculo concluído!
Margem Líquida atual média: -0.0510
Tarifa média para ML 1%: 86754.40
